In [ ]:
## IMPORTS AND FUNCTIONS ##

import requests
import pandas as pd
import json
from typing import Union
from pandas.io.json import json_normalize 
from datetime import datetime, timedelta, date
import time
import snowflake.connector
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from pathlib import Path
from snowflake.connector.pandas_tools import write_pandas

pd.set_option('display.max_columns',None)

# --- API Configuration ---
CBBD_API_KEY = "m6gcdM/RJpxwYqW6wb9008+jZ31S4YatykxZoUcgAVwFi04B5DU0mCh8T7yhj4us"
BASE_URL = "https://api.collegebasketballdata.com/"

# Snowflake Connection Details
#### THIS IS NOT WORKING DUE TO PACKAGE INSTALLATION ISSUE ####
SNOWFLAKE_USER = "DBETCHER"
SNOWFLAKE_PASSWORD = ""
SNOWFLAKE_ACCOUNT = "MIVLTKZ-DXB55236"
SNOWFLAKE_WAREHOUSE = "COMPUTE_WH"
SNOWFLAKE_DATABASE = "CBB_DATA"
SNOWFLAKE_SCHEMA = "LAKE"

with open("C:/Users/dbetc/.venv/sf_cbb_key.p8", "rb") as key:
    p_key_content = serialization.load_pem_private_key(
    key.read(),
    password= b"cbbsnowflakekey",
    backend=default_backend()
)
   
pkb = p_key_content.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

conn = snowflake.connector.connect(
            user=SNOWFLAKE_USER,
            account=SNOWFLAKE_ACCOUNT,
            private_key= pkb,          # Pass the content of the private key file
            # passphrase=b'cbbsnowflakekey',      # Pass the passphrase used to encrypt the private key
            warehouse=SNOWFLAKE_WAREHOUSE,
            database=SNOWFLAKE_DATABASE,
            schema=SNOWFLAKE_SCHEMA
        )
cursor = conn.cursor()

def fetch_cbbd_data(endpoint: str, params: dict = None, debug_raw_response: bool = False) -> Union[pd.DataFrame, None]:
    """
    Fetches data from the CollegeBasketballData.com API and returns a pandas DataFrame.

    Args:
        endpoint (str): The specific API endpoint (e.g., 'conferences', 'teams').
        params (dict): Optional dictionary of query parameters (e.g., {'year': 2023}).
        debug_raw_response (bool): If True, prints the full raw JSON response.

    Returns:
        pd.DataFrame or None: A DataFrame containing the data, or None if the request failed.
    """
    # Construct the full URL
    url = f"{BASE_URL}{endpoint}"

    # Headers are required for authentication
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {CBBD_API_KEY}' 
    }

    # Print the request URL including parameters for clarity
    print(f"--- Attempting to fetch data from endpoint: {url} (Parameters: {params}) ---")
    
    try:
        # Step 1: Make the request. The 'params' dictionary is handled automatically here.
        response = requests.get(url, headers=headers, params=params, timeout=15)
        
        # Step 2: Check for standard HTTP errors (4xx, 5xx)
        response.raise_for_status() 
        
        # Step 3: Attempt to parse JSON
        try:
            full_response_json = response.json()
        except json.JSONDecodeError:
            print("[ERROR] Failed to decode JSON response.")
            print(f"        Response Status Code: {response.status_code}")
            print("        --- RAW RESPONSE TEXT (The HTML for the Swagger UI was received instead of JSON) ---")
            # CRITICAL: Print the raw text content to diagnose non-JSON errors
            print(response.text[:500] + ('...' if len(response.text) > 500 else ''))
            print("        -------------------------------------------------------------------------------------")
            return None # Stop processing, since we can't parse the data


        # Step 4: Handle different JSON structures (FIX APPLIED HERE)
        if debug_raw_response:
            # Pretty print the entire raw JSON response for inspection
            print("\n--- RAW API RESPONSE (DEBUG) ---")
            print(json.dumps(full_response_json, indent=4))
            print("--------------------------------\n")
            
        # Check if the response is a dictionary and contains a 'data' key
        if isinstance(full_response_json, dict) and 'data' in full_response_json:
            data = full_response_json['data']
        # Check if the response is already a list (which caused the error)
        elif isinstance(full_response_json, list):
            data = full_response_json
        else:
            print("[WARNING] API response was successful but contained unexpected structure (neither dict with 'data' nor list).")
            # If it's an empty dict, etc., data will be None
            data = None 

        if data:
            df = pd.DataFrame(data)
            print(f"[SUCCESS] Successfully fetched {len(df)} records.")
            return df
        else:
            print("[WARNING] No data found in the response.")
            return None

    except requests.exceptions.HTTPError as e:
        print(f"[ERROR] HTTP Error accessing CBBD API: {e}")
        # Print specific status codes for common issues
        if response.status_code == 401:
            print("        401 Unauthorized: Check if your API key is correct and valid.")
        elif response.status_code == 429:
            print("        429 Too Many Requests: You may have hit the rate limit.")
    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Request failed: {e}")
        print("        Check your internet connection or the URL configuration.")
    except Exception as e:
        # Keeping this for general unhandled exceptions
        print(f"[ERROR] An unexpected error occurred: {e}") 
        
    return None

def write_dataframe_to_snowflake(
    df: pd.DataFrame, 
    table_name: str, 
    conn: snowflake.connector.SnowflakeConnection,
    database_name: str = SNOWFLAKE_DATABASE,
    schema_name: str = SNOWFLAKE_SCHEMA,
    chunk_size: int = 10000
):
    """
    Writes a Pandas DataFrame to a specified Snowflake table using the 
    optimized write_pandas method.
    
    This function handles staging the data in an internal Snowflake stage
    and then running a COPY INTO command, which is highly efficient.
    
    Args:
        df (pd.DataFrame): The DataFrame to write.
        table_name (str): The name of the target table (e.g., 'TEAM_ROSTERS').
        conn (snowflake.connector.SnowflakeConnection): The active connection object.
        database_name (str): Target database.
        schema_name (str): Target schema.
        chunk_size (int): Number of rows per batch for staging.
    """
    
    print(f"Attempting to write {len(df)} rows to {database_name}.{schema_name}.{table_name}...")
    
    # 1. Ensure column names meet Snowflake's standards (uppercase, no spaces/special chars)
    df.columns = [col.upper().replace(' ', '_') for col in df.columns]

    # 2. Execute the optimized write_pandas command
    try:
        # write_pandas automatically manages staging and COPY INTO
        success, nchunks, nrows, output = write_pandas(
            conn,
            df,
            table_name.upper(), # Target table name
            database=database_name.upper(),
            schema=schema_name.upper(),
            chunk_size=chunk_size,
            auto_create_table=True, # Recommended: Creates the table if it doesn't exist
            overwrite=False          # Set to True to truncate/overwrite existing data
        )
        
        if success:
            print(f"Successfully loaded {nrows} rows in {nchunks} chunks.")
            # 'output' contains the result of the COPY INTO command
            # print(f"COPY INTO result summary: {output}")
        else:
            print("WARNING: write_pandas did not return a success flag.")
            
    except Exception as e:
        print(f"FATAL ERROR during data loading to Snowflake: {e}")



In [ ]:
# Get most recent load dates/values from warehouse
team_list = pd.read_sql_query('select distinct team_name from cbb_data.lake.team_roster',conn)
team_list = team_list.iloc[:, 0].tolist() 

team_list_power = pd.read_sql_query("select team_name from cbb_data.views.dim_team where conference in ('ACC','Big 12','Big East','Big Ten','Mountain West','SEC','WCC')",conn)
team_list_power = team_list_power.iloc[:, 0].tolist()

recent_game_loaded = pd.read_sql_query('select distinct max(game_timestamp) from cbb_data.lake.games',conn)
recent_game_loaded = recent_game_loaded['MAX(GAME_TIMESTAMP)'].iloc[0]

recent_play_loaded = pd.read_sql_query('select max(game_timestamp) from cbb_data.lake.plays p inner join cbb_data.lake.games g on p.game_id=g.game_id',conn)
recent_play_loaded = recent_play_loaded['MAX(GAME_TIMESTAMP)'].iloc[0]

In [ ]:
### THIS IS FOR EXTRACTING ALL PLAYERS AND TEAMS FOR THE SEASON ###
### Only needed at beginning of season ###

# team_rosters = []

# for team in team_list:
#     item_id = team
#     enriched = fetch_cbbd_data('teams/roster',
#         {
#         "season": 2026,
#         'team': item_id
#         }
#     )
#     enriched['item_id'] = item_id
#     team_rosters.append(enriched)

# team_rosters_df = pd.concat(team_rosters,ignore_index=True)
# team_rosters_df_remove_na = team_rosters_df[team_rosters_df['players'].notna().copy()]
# team_rosters_df_remove_na = team_rosters_df_remove_na[team_rosters_df_remove_na['players'].str.len() > 0]
# team_fields_to_keep = team_rosters_df_remove_na.explode('players')

# team_fields_to_keep = team_fields_to_keep[['team','conference','season','teamId','players']]
# full_roster_members = json_normalize(team_fields_to_keep['players'])

# final_roster_df = pd.concat(
#     [team_fields_to_keep.reset_index(drop=True), 
#      full_roster_members.reset_index(drop=True)], 
#     axis=1
# )

# final_roster_subset = final_roster_df[['team','conference','season',
#         'teamId','id','name','height','weight',
#         'jersey','position']]

# final_roster_subset.rename(columns=
#     {
#         'team': 'team_name',
#         'teamId': 'team_id',
#         'id': 'player_id',
#         'name': 'player_name'
#     },
#     inplace=True)

# final_roster_subset.to_csv('C:/Users/dbetc/Downloads/cbbd_team_roster_2026.csv',index=False)

In [ ]:
### GET GAME DETAILS FOR SEASON ###
games_df = fetch_cbbd_data('games',
            {
            "season": 2026,
            "status": 'final'
            ## when using timestamp note that the API game times are 6 hours ahead of CST
            # "startDateRange": recent_game_loaded + timedelta(days=1),
            # "endDateRange": date.today() - timedelta(days=1)
            }
        )

games_subset = games_df[['id','season','seasonType','tournament','startDate',
                        'neutralSite','conferenceGame','status','gameNotes',
                        'homeTeamId','awayTeamId','homePoints','homePeriodPoints',
                        'homeWinner','awayPoints','awayPeriodPoints','excitement'
                        ]]

games_subset.rename(columns=
    {
        'id': 'game_id',
        'seasonType': 'season_type',
        'startDate': 'game_timestamp',
        'neutralSite': 'neutral_site_indicator',
        'conferenceGame': 'conference_game_indicator',
        'status': 'current_game_status',
        'gameNotes': 'special_game_note',
        'homeTeamId': 'home_team_id',
        'awayTeamId': 'away_team_id',
        'homePoints': 'home_team_points_scored',
        'homePeriodPoints': 'home_team_points_half_split',
        'homeWinner': 'home_team_victory_indicator',
        'awayPoints': 'away_team_points_scored',
        'awayPeriodPoints': 'away_team_points_half_split',
        'excitement': 'excitement_game_score'
 
    },
    inplace=True)
games_subset.to_csv('C:/Users/dbetc/Downloads/cbbd_games_list.csv',index=False)

In [ ]:
### Special one time run to catch up on dates ###
date_list = [
    # '2025-12-06 00:00:00' can't get this one to work, need NoneType handling for some erroneous game
    #did 11/3-30, 12/1-28
    ]

for date in date_list:
    plays_day_df = fetch_cbbd_data('plays/date',
                {
                "date": date
                }
            )

    plays_day_df_subset = plays_day_df[['gameId','id','playType','teamId','isHomeTeam','opponentId','homeScore','awayScore',
                            'homeWinProbability','period','clock','secondsRemaining','scoringPlay','shootingPlay',
                            'scoreValue','playText','participants','shotInfo','onFloor']]

    plays_day_df_subset.rename(columns=
        {
            'gameId': 'game_id',
            'id': 'play_id',
            'playType': 'play_type',
            'teamId': 'team_id',
            'isHomeTeam': 'home_team_indicator',
            'opponentId': 'opponent_id',
            'homeScore': 'home_team_score',
            'awayScore': 'away_team_score',
            'homeWinProbability': 'home_team_win_probability',
            'period': 'period_number',
            'clock': 'game_clock_time',
            'secondsRemaining': 'total_seconds_remaining_in_game',
            'scoringPlay': 'score_occurred_indicator',
            'shootingPlay': 'shot_attempt_indicator',
            'scoreValue': 'shot_attempt_score_value',
            'playText': 'play_description',
            'participants': 'play_participants_list',
            'shotInfo': 'shooter_id_list',
            'onFloor': 'players_on_court_list'
        },
        inplace=True)

    plays_day_df_subset.to_csv('C:/Users/dbetc/Downloads/cbbd_plays_list'+date[:10]+'.csv',index=False)

In [ ]:

# write_pandas(conn=conn,df=games_subset,table_name='GAMES',database=SNOWFLAKE_DATABASE,schema=SNOWFLAKE_SCHEMA)
## this needs fixing on missing dependency 
# write_dataframe_to_snowflake(games_subset,'GAMES',conn,SNOWFLAKE_DATABASE,SNOWFLAKE_SCHEMA,10000)
# games_subset.to_sql(
#                 name='GAMES'.upper(),
#                 con=conn, # Uses the snowflake.connector object directly
#                 schema=SNOWFLAKE_SCHEMA.upper(),
#                 if_exists='append',
#                 index=False, # We don't want the DataFrame index in the table
#                 method=None, # Ensure row-by-row insertion (avoids pyarrow/batching)
#                 chunksize=10000 # This now only affects logging/transaction size
#             )


In [ ]:
### THIS IS FOR EXTRACTING ALL LINEUP COMBINATIONS ###
## Calls each team individually ##
# Rethink this as it is too long of a call for all teams one by one - maybe by conference? #

team_lineups = []

for team in team_list_power:
    item_id = team
    enriched = fetch_cbbd_data('lineups/team',
            {
                "season": 2026,
                "team": item_id
            }
        )
    if enriched is not None:
        enriched['item_id'] = item_id
        team_lineups.append(enriched)
    else:
        # Log a warning so you know which team failed
        print(f"[WARNING] Skipping team '{item_id}' due to fetch failure or empty data.")

team_lineups_df = pd.concat(team_lineups,ignore_index=True)
team_lineups_df.to_csv('C:/Users/dbetc/Downloads/cbbd_team_lineup_results.csv',index=False)

In [ ]:
### THIS IS FOR EXTRACTING TEAM STAT DETAILS PER GAME ###
game_details_df = []

enriched = fetch_cbbd_data('games/teams',
    {
    "season": 2026,
    "startDateRange": '2025-11-29 00:00:00',
    "endDateRange": '2025-12-28 00:00:00'
    ## completed 11/3-12/28
    }
)
game_details_df.append(enriched)

game_details_df = pd.concat(game_details_df,ignore_index=True)
game_stats_fields_to_keep = game_details_df[['gameId','teamId','opponentId',
        'pace','teamStats','opponentStats'
        ]]

game_stats_fields_to_keep.rename(columns=
    {
        'gameId': 'game_id',
        'teamId': 'team_id',
        'opponentId': 'opponent_id',
        'teamStats': 'team_stats_json',
        'opponentStats': 'opponent_stats_json'
    },
    inplace=True)

# game_stats_fields_to_keep
game_stats_fields_to_keep.to_csv('C:/Users/dbetc/Downloads/cbbd_game_stats_2026.csv',index=False)